In [2]:
import os
import boto3
import time
import math
import ctypes
import pandas as pd
import subprocess
import re

# dask/parallelization libraries
import coiled
import dask
from dask.distributed import Client, LocalCluster
from dask.distributed import print as dask_print
import distributed

# scipy basics
import numpy as np
import rasterio
import rasterio.transform
import rasterio.windows
#import geopandas as gpd
import pandas as pd
import rioxarray
import xarray as xr
from rioxarray.merge import merge_arrays

from numba import jit
import concurrent.futures

from osgeo import gdal

### Creating cloud and local clusters

In [1]:
# Full cluster
coiled_cluster = coiled.Cluster(
    n_workers=40,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="10 minutes",
    region="us-east-1",
    name="AFOLU_flux_model", 
    account='jterry64', # Necessary to use the AWS environment that Justin set up in Coiled
    worker_memory = "64GiB" 
)

# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

NameError: name 'coiled' is not defined

In [6]:
# Test cluster
coiled_cluster = coiled.Cluster(
    n_workers=100,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="10 minutes",
    region="us-east-1",
    name="AFOLU_flux_model", 
    #account='wri-forest-research', # Necessary to use the AWS environment that Justin set up in Coiled
    # test using justin's account
    account='wri-forest-research', # Necessary to use the AWS environment that Justin set up in Coiled
    worker_memory = "16GiB" 
)

# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

Output()

<Client: 'tls://10.0.95.212:8786' processes=100 threads=400, memory=1.44 TiB>

In [3]:
# Local single-process cluster (local run). Will run .compute() on just one process, not a whole cluster.
local_client = Client(processes=False)
local_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.19.200.29:8787/status,
Dashboard: http://172.19.200.29:8787/status,Workers: 1
Total threads: 12,Total memory: 15.46 GiB
Status: running,Using processes: False
Comm: inproc://172.19.200.29/11700/1,Workers: 1
Dashboard: http://172.19.200.29:8787/status,Total threads: 12
Started: Just now,Total memory: 15.46 GiB
Comm: inproc://172.19.200.29/11700/4,Total threads: 12
Dashboard: http://172.19.200.29:41439/status,Memory: 15.46 GiB
Nanny: None,


In [ ]:
local_client = Client()
local_client

In [ ]:
# Local cluster with multiple workers
local_cluster = LocalCluster()  
local_client = Client(local_cluster)
local_client

### Shutting down clusters

In [ ]:
coiled_client.restart() 

In [ ]:
coiled_cluster.shutdown()

In [ ]:
local_client.shutdown()